# Projet Final Apache Spark

**Nom Etudiant :**  Djibril

**Prenom Etudiant:**  DIALLO

**Classe :**  Big Data Analytics


## Description
Ce projet consiste à utiliser Apache Spark pour faire l'analyse et le traitement des données de **[San Francisco Fire Department Calls ](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)** afin de fournir quelques KPI (*Key Performance Indicator*). Le **SF Fire Dataset** comprend les réponses aux appels de toutes les unités d'incendie. Chaque enregistrement comprend le numéro d'appel, le numéro d'incident, l'adresse, l'identifiant de l'unité, le type d'appel et la disposition. Tous les intervalles de temps pertinents sont également inclus. Étant donné que ce Dataset est basé sur les réponses et que la plupart des appels impliquent plusieurs unités, ainsi il existe plusieurs enregistrements pour chaque numéro d'appel. Les adresses sont associées à un numéro de bloc, à une intersection ou à une boîte d'appel, et non à une adresse spécifique.

**Plus de details sur la description des données cliquer sur ce [lien](https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3)**

## Travail à faire.
L'objectif de ce projet est de comprendre le **SF Fire Dataset** afin de bien répondre aux questions en utilisant les codes Spark/Scala adéquates.

- Créer un repos git (public ou privé) et partager le repos avec mon mail (limahin10@gmail.com)
- Ecrire un code lisible et bien indenté 
- N'oublier pas de mettre en commentaire la justification de vos réponses sur les cellules Markdown. 


## Note:
- Le projet est personnel, c'est-à-dire chaque notebook ne concerne qu'un seul étudiant. 
- Deadline : **Jeudi 10 janvier 2021** (Aucune de dérogation ne sera acceptée)

### Chargement des données

Importation des packages Spark

In [1]:
import org.apache.spark.sql.types._ 
import org.apache.spark.sql.functions._ 
import spark.implicits._

Intitializing Scala interpreter ...

Spark Web UI available at http://10.0.2.15:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1627762650045)
SparkSession available as 'spark'


import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import spark.implicits._


Nous allons jeter un coup d'oeil sur la structure des données avant de définir un schéma

In [2]:
!head -1 "datasets/sf-fire/sf-fire-calls.csv"

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,CallFinalDisposition,AvailableDtTm,Address,City,Zipcode,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumAlarms,UnitType,UnitSequenceInCallDispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhood,Location,RowID,Delay



Vu que la taille de ces données est énormes, inferer le schema pour un très grande volumes de données s'avère un peu couteux. Nous allons ainsi définir un schema pour le Dataset.

In [3]:
val fireSchema = StructType(Array(StructField("CallNumber", IntegerType, true),
  StructField("UnitID", StringType, true),
  StructField("IncidentNumber", IntegerType, true),
  StructField("CallType", StringType, true),                  
  StructField("CallDate", StringType, true),      
  StructField("WatchDate", StringType, true),
  StructField("CallFinalDisposition", StringType, true),
  StructField("AvailableDtTm", StringType, true),
  StructField("Address", StringType, true),       
  StructField("City", StringType, true),       
  StructField("Zipcode", IntegerType, true),       
  StructField("Battalion", StringType, true),                 
  StructField("StationArea", StringType, true),       
  StructField("Box", StringType, true),       
  StructField("OriginalPriority", StringType, true),       
  StructField("Priority", StringType, true),       
  StructField("FinalPriority", IntegerType, true),       
  StructField("ALSUnit", BooleanType, true),       
  StructField("CallTypeGroup", StringType, true),
  StructField("NumAlarms", IntegerType, true),
  StructField("UnitType", StringType, true),
  StructField("UnitSequenceInCallDispatch", IntegerType, true),
  StructField("FirePreventionDistrict", StringType, true),
  StructField("SupervisorDistrict", StringType, true),
  StructField("Neighborhood", StringType, true),
  StructField("Location", StringType, true),
  StructField("RowID", StringType, true),
  StructField("Delay", FloatType, true)))

fireSchema: org.apache.spark.sql.types.StructType = StructType(StructField(CallNumber,IntegerType,true), StructField(UnitID,StringType,true), StructField(IncidentNumber,IntegerType,true), StructField(CallType,StringType,true), StructField(CallDate,StringType,true), StructField(WatchDate,StringType,true), StructField(CallFinalDisposition,StringType,true), StructField(AvailableDtTm,StringType,true), StructField(Address,StringType,true), StructField(City,StringType,true), StructField(Zipcode,IntegerType,true), StructField(Battalion,StringType,true), StructField(StationArea,StringType,true), StructField(Box,StringType,true), StructField(OriginalPriority,StringType,true), StructField(Priority,StringType,true), StructField(FinalPriority,IntegerType,true), StructField(ALSUnit,BooleanType,true)...


In [4]:
val sfFireFile = "datasets/sf-fire/sf-fire-calls.csv"
val fireDF = spark
  .read
  .schema(fireSchema)
  .option("header", "true")
  .csv(sfFireFile)

sfFireFile: String = datasets/sf-fire/sf-fire-calls.csv
fireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]


Nous allons mettre en cache le Dataframe

In [5]:
fireDF.cache()

res0: fireDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


In [6]:
fireDF.count()

res1: Long = 175296


In [7]:
fireDF.printSchema()

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- IncidentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = true)
 

In [8]:
fireDF.show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+------------

Filtrage des d'appels de type "Medical Incident"

In [9]:
val fewFireDF = fireDF
  .select("IncidentNumber", "AvailableDtTm", "CallType") 
  .where($"CallType" =!= "Medical Incident")

fewFireDF.show(5, false)

fewFireDF: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [IncidentNumber: int, AvailableDtTm: string ... 1 more field]


+--------------+----------------------+--------------+
|IncidentNumber|AvailableDtTm         |CallType      |
+--------------+----------------------+--------------+
|2003235       |01/11/2002 01:51:44 AM|Structure Fire|
|2003250       |01/11/2002 04:16:46 AM|Vehicle Fire  |
|2003259       |01/11/2002 06:01:58 AM|Alarms        |
|2003279       |01/11/2002 08:03:26 AM|Structure Fire|
|2003301       |01/11/2002 09:46:44 AM|Alarms        |
+--------------+----------------------+--------------+
only showing top 5 rows



### Question 1
**Combien de types d'appels distincts ont été passés ?**  
Pour être sûr, il ne faut pas compter les valeurs «nulles» dans la colonne.

In [11]:
// Reponse 1

val nbre_appel_distinct = fireDF.select(countDistinct("CallType") as "Number of CallType distinct")
nbre_appel_distinct.show()

+---------------------------+
|Number of CallType distinct|
+---------------------------+
|                         30|
+---------------------------+



nbre_appel_distinct: org.apache.spark.sql.DataFrame = [Number of CallType distinct: bigint]


### Question 2

**Quels types d'appels différents ont été passés au service d'incendie?**

In [12]:
// Reponse 2

val appels_differnts = fireDF.filter($"CallFinalDisposition" === "Fire")
                        .select("CallType").distinct

appels_differnts.show()

+--------------------+
|            CallType|
+--------------------+
|Elevator / Escala...|
|         Marine Fire|
|  Aircraft Emergency|
|Confined Space / ...|
|      Administrative|
|              Alarms|
|Odor (Strange / U...|
|Citizen Assist / ...|
|              HazMat|
|Watercraft in Dis...|
|           Explosion|
|           Oil Spill|
|        Vehicle Fire|
|  Suspicious Package|
|Extrication / Ent...|
|               Other|
|        Outside Fire|
|   Traffic Collision|
|       Assist Police|
|Gas Leak (Natural...|
+--------------------+
only showing top 20 rows



appels_differnts: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string]


### Question 3

**Trouver toutes les réponses ou les délais sont supérieurs à 5 minutes?**

*Indication
1. Renommer la colonne Delay -> ReponseDelayedinMins
2. Retourner un nouveau DataFrame
3. Afficher tous les appels où le temps de réponse à un site d'incendie a eu lieu après un retard de plus de 5 minutes

In [15]:
// Reponse 3
val newFireDF = fireDF.withColumnRenamed("Delay", "ResponseDelayedinMins")

val delai = newFireDF.filter($"ResponseDelayedinMins" > 5)
                    .select("CallNumber", "CallType", "ResponseDelayedinMins")

delai.show()

+----------+--------------------+---------------------+
|CallNumber|            CallType|ResponseDelayedinMins|
+----------+--------------------+---------------------+
|  20110315|    Medical Incident|                 5.35|
|  20120147|    Medical Incident|                 6.25|
|  20130013|    Medical Incident|                  5.2|
|  20140067|Citizen Assist / ...|                  5.6|
|  20140177|    Medical Incident|                 7.25|
|  20150056|    Medical Incident|            11.916667|
|  20150254|    Medical Incident|             5.116667|
|  20150265|    Medical Incident|             8.633333|
|  20150265|    Medical Incident|             95.28333|
|  20150380|    Medical Incident|                 5.45|
|  20150414|    Medical Incident|                  7.6|
|  20160059|    Medical Incident|             6.133333|
|  20160064|      Structure Fire|            5.1833334|
|  20170118|    Medical Incident|            6.9166665|
|  20170342|    Medical Incident|               

newFireDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
delai: org.apache.spark.sql.DataFrame = [CallNumber: int, CallType: string ... 1 more field]


### Transformations des dates  
Maintenant nous allons d'abord:  
1. Transformer les dates de type String en Spark Timestamp afin que nous puissions effectuer des requêtes basées sur la date plus tard    
2. Retourner le Dataframe transformée  
3. Mettre en cache le nouveau DataFrame  

In [16]:
val fireTSDF = newFireDF
  .withColumn("IncidentDate", to_timestamp(col("CallDate"), "MM/dd/yyyy")).drop("CallDate") 
  .withColumn("OnWatchDate", to_timestamp(col("WatchDate"), "MM/dd/yyyy")).drop("WatchDate") 
  .withColumn("AvailableDtTS", to_timestamp(col("AvailableDtTm"), "MM/dd/yyyy hh:mm:ss a")).drop("AvailableDtTm")

fireTSDF.cache()

fireTSDF: org.apache.spark.sql.DataFrame = [CallNumber: int, UnitID: string ... 26 more fields]
res11: fireTSDF.type = [CallNumber: int, UnitID: string ... 26 more fields]


### Question 4
**Quels sont les types d'appels les plus courants?**

In [20]:
//Reponse 4

val appels_freq = fireDF.groupBy("CallType")
                            .agg(count("*") as "Total")
                            .orderBy(desc("Total"))

appels_freq.show(10)

+--------------------+------+
|            CallType| Total|
+--------------------+------+
|    Medical Incident|113794|
|      Structure Fire| 23319|
|              Alarms| 19406|
|   Traffic Collision|  7013|
|Citizen Assist / ...|  2524|
|               Other|  2166|
|        Outside Fire|  2094|
|        Vehicle Fire|   854|
|Gas Leak (Natural...|   764|
|        Water Rescue|   755|
+--------------------+------+
only showing top 10 rows



appels_freq: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [CallType: string, Total: bigint]


### Question 5-a
**Quels sont boites postaux rencontrés dans les appels les plus courants?**

In [19]:
//Reponse 5-a

val boites_freq = fireDF.groupBy("Box")
                            .agg(count("*") as "Total")
                            .orderBy(desc("Total"))

boites_freq.show(10)

+----+-----+
| Box|Total|
+----+-----+
|2251| 1911|
|1453| 1790|
|1461| 1552|
|5236| 1269|
|2252| 1246|
|1545| 1231|
|2248| 1122|
|1456| 1090|
|1554| 1052|
|1365| 1031|
+----+-----+
only showing top 10 rows



boites_freq: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Box: string, Total: bigint]


### Question 5-a
**Quels sont les quartiers de San Francisco dont les codes postaux sont 94102 et 94103?**

In [21]:
//Reponse 5-b

val district = fireDF.filter($"City"==="San Francisco" and ($"zipcode"===94102 || $"zipcode"===94103))
                        .select("Neighborhood", "zipcode").distinct

district.show()

+--------------------+-------+
|        Neighborhood|zipcode|
+--------------------+-------+
|        Potrero Hill|  94103|
|    Western Addition|  94102|
|          Tenderloin|  94102|
|            Nob Hill|  94102|
| Castro/Upper Market|  94103|
|     South of Market|  94102|
|     South of Market|  94103|
|        Hayes Valley|  94103|
|Financial Distric...|  94102|
|         Mission Bay|  94103|
|          Tenderloin|  94103|
|Financial Distric...|  94103|
|        Hayes Valley|  94102|
|             Mission|  94103|
+--------------------+-------+



district: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Neighborhood: string, zipcode: int]


### Question 6
**Determiner le nombre total d'appels, ainsi que la moyenne, le minimum et le maximum du temps de réponse des appels?**

In [22]:
//Reponse 6
val statistiques = newFireDF.select(count("ResponseDelayedinMins") as "Nombre d'appel", 
                    avg("ResponseDelayedinMins") as "Moyenne",
                    max("ResponseDelayedinMins") as "Maximum",
                    min("ResponseDelayedinMins")as "Minimum"
)

statistiques.show

+--------------+-----------------+-------+-----------+
|Nombre d'appel|          Moyenne|Maximum|    Minimum|
+--------------+-----------------+-------+-----------+
|        175296|3.892364154521585|1844.55|0.016666668|
+--------------+-----------------+-------+-----------+



statistiques: org.apache.spark.sql.DataFrame = [Nombre d'appel: bigint, Moyenne: double ... 2 more fields]


### Question 7-a
**Combien d'années distinctes trouve t-on dans ce Dataset?**  
Dans ce dataset nous avons des données comprises entre 2000-2018. Vous pouvez utilisez la fonction Spark `year()` pour les dates en Timestamp

In [24]:
//Reponse 7-a

val diff_annee = fireTSDF.select(countDistinct(year($"IncidentDate")))

diff_annee.show()

+------------------------------------------------+
|count(DISTINCT year(CAST(IncidentDate AS DATE)))|
+------------------------------------------------+
|                                              19|
+------------------------------------------------+



diff_annee: org.apache.spark.sql.DataFrame = [count(DISTINCT year(CAST(IncidentDate AS DATE))): bigint]


### Question 7-b
**Quelle semaine de l'année 2018 a eu le plus d'appels d'incendie?**

In [25]:
//Reponse 7-b

fireTSDF.filter(year($"IncidentDate")===2018 && $"CallFinalDisposition"==="Fire")
.select(max(weekofyear($"IncidentDate"))).show

+-----------------------------+
|max(weekofyear(IncidentDate))|
+-----------------------------+
|                           45|
+-----------------------------+



### Question 8
**Quels sont les quartiers de San Francisco qui ont connu le pire temps de réponse en 2018?**

In [26]:
//Reponse 8

newFireDF.filter($"City" ==="San Francisco")
                    .select("ResponseDelayedinMins","Neighborhood")
                    .orderBy(desc("ResponseDelayedinMins")).show()

+---------------------+--------------------+
|ResponseDelayedinMins|        Neighborhood|
+---------------------+--------------------+
|              1844.55|     South of Market|
|               931.45|           Chinatown|
|            491.26666|           Chinatown|
|            406.63333|Financial Distric...|
|                405.7|   Visitacion Valley|
|            340.48334|          Tenderloin|
|               314.95|             Mission|
|            308.56668|Bayview Hunters P...|
|               297.05|      Bernal Heights|
|               239.95|Financial Distric...|
|            202.13333|Bayview Hunters P...|
|               199.45|     Sunset/Parkside|
|            175.86667|      Haight Ashbury|
|               171.85|          Tenderloin|
|                171.7|Financial Distric...|
|                155.8|Bayview Hunters P...|
|                145.2|             Mission|
|            135.51666|Financial Distric...|
|            135.03334|Financial Distric...|
|         

### Question 9

**Comment stocker les données du Dataframe sous format de fichiers Parquet?**

In [27]:
//Reponse 9

fireDF.write.parquet("data.parquet")

### Question 10
**Comment relire les données stockée en format Parquet?**

In [28]:
//Reponse 10

spark.read.parquet("data.parquet").show

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|      UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+

## FIN